In [5]:
import pandas as pd
import os


datadir='/cellar/users/snwright/Data/RareCommon'

In [114]:
rv_study_info = pd.read_csv(os.path.join(datadir, 'RAVAR', 'RVStudyInfo_manual_long.tsv'), sep='\t')
rv_study_info['rv_idx'] = [i for i in range(len(rv_study_info))]

In [115]:
rv_study_info['N'] = rv_study_info.apply(lambda x: x['N-Actual'] if x['N-Actual'] > 0 else x.N , axis=1)

In [116]:
rv_study_info['Classification'] = rv_study_info.apply(lambda x: x['Classification.1'] if isinstance(x['Classification.1'], str) else x['Classification'] , axis=1)

In [117]:
rv_study_info = rv_study_info.drop(columns=['N-Actual', 'Classification.1', 'Unnamed: 11'])

In [118]:
## Study specific
add_886 = pd.read_csv(os.path.join(datadir, 'RAVAR', '34662886_samples_clean.txt'), sep='\t')
add_796 = pd.read_csv(os.path.join(datadir, 'RAVAR', '36809768_samples.csv'))
add_343b = pd.read_csv(os.path.join(datadir, 'RAVAR', '34375979_union_binary.txt'), sep='\t')
add_343q = pd.read_csv(os.path.join(datadir, 'RAVAR', '34375979_union_quant.txt'), sep='\t')

## 343

In [119]:
add_343b['trait'] = add_343b.Phenotype.apply(lambda x: x.split(' ', 1)[-1].lower())
add_343q['trait'] = add_343q.Phenotype.apply(lambda x: x.replace(' measurement', '').lower())

In [120]:
rv_343 = rv_study_info.loc[rv_study_info.PMID==34375979]
rv_343['trait'] = rv_343['Reported Trait'].apply(lambda x: x.replace(' measurement', '').lower())

/tmp/ipykernel_2298860/2512684926.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_343['trait'] = rv_343['Reported Trait'].apply(lambda x: x.replace(' measurement', '').lower())


In [121]:
rv_343_traits = rv_343['Reported Trait'].unique()

In [122]:
add_343b[(add_343b.Path=='Union') & (add_343b.trait=='malignant neoplasm of breast')]

,Phenotype,Path,Root,Field,N Cases_300Kv2,N Controls_300Kv2,Smart Controls,Sex Matching,Unique to Release 300Kv2,trait
1292,Union#BlockC50-C50#C50-C50 Malignant neoplasm ...,Union,Chapter II Neoplasms,Union,11091,136034,NO,YES,NO,malignant neoplasm of breast
1422,Union#C50#C50 Malignant neoplasm of breast,Union,Chapter II Neoplasms,Union,10031,137079,NO,YES,NO,malignant neoplasm of breast


In [123]:
binary = rv_343.merge(add_343b[add_343b.Path=='Union'], on='trait', how='inner').sort_values('N Cases_300Kv2', ascending=False).drop_duplicates(subset='trait')
binary['N'] = binary['N Cases_300Kv2'] + binary['N Controls_300Kv2']
binary['Classification'] = 'Population CC'

In [124]:
remaining = set(rv_343_traits) - set(binary['Reported Trait'].values)

In [125]:
quant = rv_343.merge(add_343q, on='trait', how='inner').sort_values('N Tested', ascending=False).drop_duplicates(subset='trait')
quant['N'] = quant['N Tested']
quant['Classification'] = 'Population Q'

In [126]:
all_add_343 = pd.concat([binary, quant]).loc[:, rv_study_info.columns]

In [130]:
rv_study1 = pd.concat([rv_study_info[~rv_study_info.rv_idx.isin(all_add_343.rv_idx.values)], all_add_343])

## 886

In [201]:
rv_886 = rv_study1[rv_study1.PMID==34662886]
rv_886['trait'] = rv_886['Reported Trait'].apply(lambda x: x.replace(' measurement', '').replace('_inst_mean__rint', '').replace('__bin', '').replace('Rgc_', '').split('_', 1)[-1].replace('_', ' ').split(' (')[0].lower())

/tmp/ipykernel_2298860/3536213656.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_886['trait'] = rv_886['Reported Trait'].apply(lambda x: x.replace(' measurement', '').replace('_inst_mean__rint', '').replace('__bin', '').replace('Rgc_', '').split('_', 1)[-1].replace('_', ' ').split(' (')[0].lower())


In [219]:
add_886 = pd.read_csv(os.path.join(datadir, 'RAVAR', '34662886_samples_clean.txt'), sep='\t')
add_886['Ncase'] = add_886['N cases with 0|1|2 copies of effect allele'].apply(lambda x: sum([int(n) for n in x.split('|') if n != np.nan]))
add_886['Ncontrol'] = add_886['N controls with 0|1|2 copies of effect allele'].apply(lambda x: sum([int(n) for n in x.split('|') if n != 'NA']))

In [220]:
add_886['N'] = add_886['Ncase'] + add_886['Ncontrol']
add_886 = add_886.loc[:, ('Trait', 'Ncase', 'Ncontrol', 'N')].groupby('Trait').mean().reset_index()

In [221]:
add_886['Classification'] = add_886.Ncontrol.apply(lambda x: 'Population Q' if x==0 else 'Population CC')
add_886['trait'] = add_886.Trait.apply(lambda x: x.split(' (')[0].lower())

In [222]:
z = rv_886.drop(columns=['N', 'Classification']).merge(add_886, on='trait', how='inner')

In [223]:
z.head()

,Reported Trait,Trait Label,Trait Ontology id,PMID,COHORT,Ancestry,Notes,rv_idx,trait,Trait,Ncase,Ncontrol,N,Classification
0,Gamma glutamyltransferase (30730) measurement,protein-glutamine gamma-glutamyltransferase e ...,EFO:0020690,34662886,UKB,EUR,Separate File,10,gamma glutamyltransferase,Gamma glutamyltransferase (30730),411824.383333,0.0,411824.383333,Population Q
1,Apolipoprotein b (30640) measurement,apolipoprotein B change measurement,EFO:0020874,34662886,UKB,EUR,Separate File,11,apolipoprotein b,Apolipoprotein B (30640),410088.384615,0.0,410088.384615,Population Q
2,Triglycerides (30870),triglyceride,CHEBI:17855,34662886,UKB,EUR,Separate File,12,triglycerides,Triglycerides (30870),411723.153846,0.0,411723.153846,Population Q
3,30730_gamma_glutamyltransferase_inst_mean__rin...,protein-glutamine gamma-glutamyltransferase e ...,EFO:0020690,34662886,UKB,EUR,Separate File,17,gamma glutamyltransferase,Gamma glutamyltransferase (30730),411824.383333,0.0,411824.383333,Population Q
4,Hdl cholesterol (30760) measurement,HDL cholesterol change measurement,EFO:0007805,34662886,UKB,EUR,Separate File,67,hdl cholesterol,HDL cholesterol (30760),378175.226277,0.0,378175.226277,Population Q


In [207]:
missing = set(rv_886['Reported Trait'].values) - set(z['Reported Trait'].values)

In [227]:
rv_study2 = pd.concat([rv_study1[~rv_study1.rv_idx.isin(z.rv_idx.values)], z.loc[:, rv_study1.columns].sort_values('N', ascending=False).drop_duplicates(subset='rv_idx')])

## 768

In [286]:
rv_768 = rv_study2[rv_study2.PMID==36809768].drop(columns=['N', 'Classification'])
rv_768['trait'] = rv_768['Reported Trait'].apply(lambda x: x.replace(' measurement', '').lower())

In [287]:
add_768 = pd.read_csv(os.path.join(datadir, 'RAVAR', '36809768_samples.csv'))
add_768['N'] = add_768['Number with non-missing data               (out of 412,393)'].apply(lambda x: (x.replace(',', '')))

In [288]:
add_768_1 = add_768.iloc[0:102]
add_768_2 = add_768.iloc[103:]
add_768_2.columns = ['Measurement', 'Description', 'N-in', 'Adjusted', 'N']

In [289]:
add_768_1.head(0)

,Metabolite measurement,Metabolite description,"Number with non-missing data (out of 412,393)",Analysed for medication-adjusted values?,N


In [290]:
add_768_1['trait'] = add_768_1['Metabolite description'].apply(lambda x: x.lower())
add_768_2['trait'] = add_768_2['Description'].apply(lambda x: x.lower())
add_768_1['N'] = add_768_1['N'].astype(int)
add_768_2['N'] = add_768_2['N'].astype(int)

/tmp/ipykernel_2298860/806896550.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add_768_1['trait'] = add_768_1['Metabolite description'].apply(lambda x: x.lower())
/tmp/ipykernel_2298860/806896550.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add_768_2['trait'] = add_768_2['Description'].apply(lambda x: x.lower())
/tmp/ipykernel_2298860/806896550.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [291]:
match_768_1 = rv_768.merge(add_768_1, on='trait', how='inner')

In [292]:
match_768_2 = rv_768.merge(add_768_2, on='trait', how='inner')

In [293]:
all_match = pd.concat([match_768_1, match_768_2])

In [295]:
all_match['Classification'] = 'Population Q'

In [296]:
all_match = all_match.loc[:, rv_study2.columns].sort_values('N', ascending=False).drop_duplicates(subset='rv_idx')

In [297]:
rv_study3 = pd.concat([rv_study2[~rv_study2.rv_idx.isin(all_match.rv_idx.values)], all_match])

In [303]:
rv_study3.to_csv(os.path.join(datadir, 'rv_study_info_cleaned_with_manual.tsv'), sep='\t')